In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install timm

In [3]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms as ts
import torchvision as tv
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn
import timm
from PIL import Image
from tqdm import tqdm
import random
import os

%config Completer.use_jedi = False

In [4]:
def setseed(seednum = 20):
    torch.manual_seed(seednum)
    torch.cuda.manual_seed(seednum)
    torch.cuda.manual_seed_all(seednum)
    np.random.seed(seednum)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seednum)

In [5]:
setseed(35)

In [6]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
device

device(type='cuda')

# Dataset

In [7]:
!pip install git+https://github.com/ildoonet/cutmix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ildoonet/cutmix to /tmp/pip-req-build-r6b0ghho
  Running command git clone -q https://github.com/ildoonet/cutmix /tmp/pip-req-build-r6b0ghho
  Created wheel for cutmix: filename=cutmix-0.1-py3-none-any.whl size=3599 sha256=1525308fa5b2a6b2cb387552e5e110d0c8da800d4a243a518b0ea82825c1ab8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-yufcjsb9/wheels/a9/81/a7/d3822499b14d97b1e2ef7e7538b70f15355607cfc7526f7cd5
Successfully built cutmix


In [8]:
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

In [9]:
transform = ts.Compose(
    [
        ts.ToTensor(),
        ts.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [10]:
train_set = ds.CIFAR10(root='../data/', train=True, transform=transform, download=True)
train_set = CutMix(train_set, num_class=10, beta=1.0, prob=0.5, num_mix=2)

val_set = tv.datasets.CIFAR10(root='../data/', train=False, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/cifar-10-python.tar.gz to ../data/
Files already downloaded and verified


In [11]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False, num_workers=0)

In [12]:
def make_test_list(data_dir):
    
    test_img_list = list()
    
    files = os.listdir(data_dir)
    
    for i in range(10):
        img_file = data_dir + "/" + files[i]
        img = os.listdir(img_file)
        for j in range(200):
            img_path = data_dir + "/" +  files[i] + "/" + img[j]
            
            test_img_list.append(img_path)
    return test_img_list

In [13]:
img_list = make_test_list("./drive/MyDrive/Statistical_Deep_Image")

In [14]:
def make_test_label(label_list):
    
    test_label_list = list()
    
    for i in [0,2,1,3,4,5,6,7,8,9]:
        for j in range(200):
            test_label_list.append(i)
    
    return test_label_list

In [15]:
label = os.listdir("./drive/MyDrive/Statistical_Deep_Image")
img_label_list = make_test_label(label)

In [16]:
class testset(torch.utils.data.Dataset):
    def __init__(self, img_list, img_label_list, transform):
        self.file_list = img_list
        self.labels = img_label_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = self.labels[index]
        return img_transformed, label

In [17]:
test_set = testset(img_list = img_list,
                  img_label_list = img_label_list,
                  transform = transform)

In [18]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 64, shuffle=False, num_workers=0)

In [19]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model

In [20]:
model = timm.create_model('efficientnetv2_s', pretrained=False, num_classes=10).to(device)

In [28]:
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = CutMixCrossEntropyLoss(True)

# Train

In [29]:
def train(num_epoch):
    best_accuracy = 0.0 
    
    for epoch in tqdm(range(num_epoch)):

        running_train_loss = 0.0
        running_val_loss = 0.0
        true = 0
        total = 0

        for i, data in enumerate(train_loader, 0) :
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_train_loss += loss.item()
        
        with torch.no_grad():
            model.eval()
            for i, data in enumerate(val_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                predicted_outputs = model(inputs)
                val_loss = criterion(predicted_outputs, labels)
                
                _, predicted = torch.max(predicted_outputs, 1) 
                
                running_val_loss += val_loss.item()
                total += labels.size(0)
                true += (predicted == labels).sum().item()
                
        train_loss_per_epoch = running_train_loss / len(train_loader)
        val_loss = running_val_loss/len(val_loader)
        accuracy = (100 * true / total)     
        
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), 'EfficientNet_CutMix_weights.pth')
            best_accuracy = accuracy
            
        print('epoch: %d' %(epoch+1), ' train_loss: %.3f' %train_loss_per_epoch, ' val_loss: %.4f' %val_loss, ' Accuracy: %.2f %%' % (accuracy))

In [33]:
train(20)

  5%|▌         | 1/20 [01:52<35:37, 112.47s/it]

epoch: 1  train_loss: 0.994  val_loss: 0.6065  Accuracy: 81.19 %


 10%|█         | 2/20 [03:44<33:34, 111.93s/it]

epoch: 2  train_loss: 0.994  val_loss: 0.6048  Accuracy: 81.26 %


 15%|█▌        | 3/20 [05:36<31:44, 112.05s/it]

...

epoch: 18  train_loss: 0.972  val_loss: 0.5985  Accuracy: 81.51 %


 95%|█████████▌| 19/20 [35:22<01:51, 111.80s/it]

epoch: 19  train_loss: 0.981  val_loss: 0.6001  Accuracy: 81.35 %


100%|██████████| 20/20 [37:14<00:00, 111.74s/it]

epoch: 20  train_loss: 0.983  val_loss: 0.5991  Accuracy: 81.37 %


# Test

In [34]:
model.load_state_dict(torch.load('EfficientNet_CutMix_weights.pth'))
model.eval()

EfficientNet(
  (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Seq

In [35]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' % (100 * correct / total))

100%|██████████| 32/32 [00:03<00:00,  8.02it/s]

Accuracy: 8 %
